## Proyecto 9


La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes
utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el
comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:
Smart o Ultra.
Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto.
Para ayudarte a entrenar tu modelo, utilizarás los datos de comportamiento del usuario
del curso sobre Análisis estadísticos de datos. Como ya hiciste el paso de procesar los
datos, puedes lanzarte directo a crear el modelo.
Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de
exactitud es 0.75. Usa el dataset para comprobar la exactitud.

In [3]:
#Examinar los datos
import pandas as pd

In [4]:
#Leer el dataset
df = pd.read_csv('users_behavior.csv')

In [5]:
#Mostrar informacion
def info_data(datos):
    print('Primeras Filas')
    print(datos.head())
    print('Informacion General')
    print(datos.info())
    print('Datos nulos')
    print(datos.isna().sum())
    print('Datos Duplicados')
    print(datos.duplicated().sum())

In [6]:
#Visualizacion de datos
info_data(df)

Primeras Filas
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
Informacion General
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
Datos nulos
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64
Datos Duplicados
0


In [7]:
#Visualizar los datos
print(df.shape)
print(df.head(5))

(3214, 5)
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


## Observaciones
Para analizar el comportamiento de los clientes para poder recomendar uno de los planes Smart o ultra el objetivo  que se va seleccionar es la columna is_ultra ya que esta estandarizada es 0 si el plan es Samrt y 1 si el Plan que tiene el usuario es Ultra y los atributos son todos los datos de las columnas que permitar analizar el comportamiento de los usuarios como la cantidad de llamadas, minutos, mensajes y megabytes. Es importante considerar que los datos son variables numericas y para poder elegir un modelo que prediga cual es el mejor plan para ese usuario se tendrian que estandarizar esos datos y al ser un modelo de clasificacion se va utilizar un algoritmo de regresion logistica. 

## Segmentacion de datos

In [10]:
#Importar todas las herramientas de la libreria que se va utilizar
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [11]:
df.reset_index(drop=True, inplace=True)

In [16]:
# Separar los datos en características (features) y objetivo (target)
features = df.drop(columns=['is_ultra'])
target = df['is_ultra']

In [46]:
# Dividir los datos en conjunto de entrenamiento y conjunto de validación
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=54321)
print(target_train.shape)
print(features_train.shape)
print(target_valid.shape)
print(features_valid.shape)

(2410,)
(2410, 4)
(804,)
(804, 4)


In [48]:
#Definir las variables a escalar
numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
#Definir y ajustar un modelo de escalonamiento
scaler= StandardScaler()
scaler.fit_transform(features_train[numeric_features])

array([[ 0.32879871,  0.4360359 , -0.59962698,  0.37998073],
       [-0.03409328,  0.14618617, -0.95710064, -0.3416271 ],
       [ 0.08687072, -0.03850897, -0.51713306, -0.68225856],
       ...,
       [-0.51794926, -0.33508344, -0.73711685, -0.5952518 ],
       [-0.66915426, -0.83275803,  0.00532845, -2.27698731],
       [ 0.17759372,  0.3154615 , -0.6821209 , -0.3441867 ]])

In [50]:
#Estandarizar los valores
features_train[numeric_features]=scaler.transform(features_train[numeric_features])
features_valid[numeric_features]=scaler.transform(features_valid[numeric_features])

In [59]:
#Crear modelo y entrenar 
model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train)

LogisticRegression(random_state=54321, solver='liblinear')

In [54]:
#Realizar predicciones
predictions = model.predict(features_valid)

In [61]:
#Evaluar el modelo 
score_train = model.score(features_train, target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7568464730290456
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7251243781094527


In [63]:
#Visualizar el reporte de Clasificacion
report = classification_report(target_valid, predictions)
print("Reporte de clasificación:\n", report)

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.72      0.98      0.83       545
           1       0.82      0.19      0.31       259

    accuracy                           0.73       804
   macro avg       0.77      0.58      0.57       804
weighted avg       0.75      0.73      0.66       804



### Observaciones
Para el conjunto de datos el mejor modelo de prediccion es el de regresion logistica porque es el mas preciso y mas veloz.  

### Tarea Adiconal 

In [69]:
# Datos triviales para prueba de cordura
check_data = {
    'calls': [50, 100],
    'minutes': [300, 700],
    'messages': [50, 80],
    'mb_used': [10000, 20000]
}

check_df = pd.DataFrame(check_data)

# Transformar los datos de la prueba de cordura utilizando el escalador ajustado
check_df[numeric_features] = scaler.transform(sanity_check_df[numeric_features])

# Realizar predicciones con el modelo
check_predictions = model.predict(check_df)

print("Predicciones de la prueba de cordura:", sanity_check_predictions)

Predicciones de la prueba de cordura: [0 1]
